In [73]:
import cv2 
import os 
from PIL import Image
import numpy as np 
import pickle
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from EnsembleModelTraining import load_dataset, transform_data, split_data

In [75]:
model = load_model("KERAS MODELS/MRV_Inception.keras")

with open("LabelEncoderData.pkl", "rb") as f:
    le = pickle.load(f)
    

c:\Users\Alaric\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [66]:
GET_ROI_TRANSFORMATIONS = False
ROI_CW = 8

In [67]:
def load_image(joined_path):
    im = Image.open(joined_path)
    im = np.array(im)
    im = Image.fromarray(im)

    return im

def get_dimensions(im):
    w, h = im.size
    w_steps = w // ROI_CW
    h_steps = h // ROI_CW
    w_residual = w - w_steps * ROI_CW
    h_residual = h - h_steps * ROI_CW

    return w, h, w_steps, h_steps, w_residual, h_residual

def resize_normalize_image(im, size=(64, 64)):
    image = np.array(im) 
    image = cv2.resize(image, (64, 64))
    image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    return image 
    
def transform_image(im, w_steps, h_steps, w_residual, h_residual):
    
    proposed_images = [] 

    for x in range(0, ROI_CW):
        for y in range(0, ROI_CW):
            for i in range(x + 1, ROI_CW + 1):
                for j in range(y + 1, ROI_CW + 1):
                    _left = x * w_steps
                    _top = y * h_steps
                    _right = (i) * w_steps
                    _bottom = (j) * h_steps
                    _right += 0 if i < ROI_CW - 1 else w_residual
                    _bottom += 0 if j < ROI_CW - 1 else h_residual

                    cropped_image = im.crop((_left, _top, _right, _bottom))
                    cropped_image = resize_normalize_image(cropped_image)
                    
                    proposed_images.append(cropped_image)
                    
    proposed_images = np.array(proposed_images)
    return proposed_images

def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result


In [74]:
_X, _y = load_dataset()
_y = transform_data(_y)
X_train, X_valid, X_test, y_train, y_valid, y_test = split_data(_X, _y)

In [76]:
model.evaluate(X_train, y_train)

2452/2452 [==============================] - 55s 21ms/step - loss: 0.0010 - accuracy: 0.9997


[0.0010058225598186255, 0.999732255935669]

In [77]:
model.evaluate(X_valid, y_valid)

273/273 [==============================] - 8s 30ms/step - loss: 0.0022 - accuracy: 0.9997


[0.0021524219773709774, 0.9996557235717773]

In [78]:
model.evaluate(X_test, y_test)

681/681 [==============================] - 14s 21ms/step - loss: 7.1032e-04 - accuracy: 1.0000


[0.0007103223470039666, 0.999954104423523]

In [80]:
orig_folder = "Segmented Herbal Leaf Images"

total_plant_class_tally = {}

for plant_class in os.listdir(orig_folder):
    print(plant_class)
    plant_class_predictions = {}

    for testing_image in os.listdir(os.path.join(orig_folder, plant_class)):
        try:
            im = load_image(os.path.join(orig_folder, plant_class, testing_image))
            w, h, w_steps, h_steps, w_residual, h_residual = get_dimensions(im)
            
            if GET_ROI_TRANSFORMATIONS:
                proposed_images = transform_image(im, w_steps, h_steps, w_residual, h_residual)
            
            else:
                proposed_images = np.array([resize_normalize_image(im, (64, 64))])


            _im = proposed_images[0]
            _im = np.array(_im)

            _im = rotate_image(_im, 36)
            # cv2.imshow("hatdog", _im)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()

            raw_y = model.predict(proposed_images)
            raw_y = np.array(raw_y)
            pred_y = np.argmax(raw_y, axis=1)
            predict_dict = {}
            for i in pred_y:
                predict_dict[i] = predict_dict.get(i, 0) + 1

            transformed_dict = {}
            for predict in predict_dict:
                transformed_dict[le.classes_[predict]] = predict_dict[predict] / len(pred_y)

            x = list(dict(sorted(transformed_dict.items(), key=lambda x:x[1], reverse=True)).items())[0]
            
            plant_class_predictions[testing_image] = x
        
        except Exception as E:
            print("Error at:", testing_image, ":")

    print(plant_class_predictions)

    plant_class_tally = {}

    for (file_name, (item_name, confidence)) in plant_class_predictions.items():
        plant_class_tally[item_name] = plant_class_tally.get(item_name, 0) + 1

    total_plant_class_tally[plant_class] = plant_class_tally
    


Artocarpus Heterophyllus (Jackfruit)
1/1 [==============================] - 0s 28ms/step
Error at: 14.jpg :
1/1 [==============================] - 0s 54ms/step
{'1.jpg': ('Moringa Oleifera (Malunggay)', 1.0), '10.jpg': ('Moringa Oleifera (Malunggay)', 1.0), '11.jpg': ('Vitex Negundo (Lagundi)', 1.0), '12.jpg': ('Moringa Oleifera (Malunggay)', 1.0), '13.jpg': ('Moringa Oleifera (Malunggay)', 1.0), '15.jpg': ('Moringa Oleifera (Malunggay)', 1.0), '16.jpg': ('Moringa Oleifera (Malunggay)', 1.0), '17.jpg': ('Moringa Oleifera (Malunggay)', 1.0), '18.jpg': ('Moringa Oleifera (Malunggay)', 1.0), '19.jpg': ('Moringa Oleifera (Malunggay)', 1.0), '2.jpg': ('Moringa Oleifera (Malunggay)', 1.0), '20.jpg': ('Moringa Oleifera (Malunggay)', 1.0), '21.jpg': ('Vitex Negundo (Lagundi)', 1.0), '22.jpg': ('Moringa Oleifera (Malunggay)', 1.0), '23.jpg': ('Moringa Oleifera (Malunggay)', 1.0), '24.jpg': ('Moringa Oleifera (Malunggay)', 1.0), '25.jpg': ('Moringa Oleifera (Malunggay)', 1.0), '26.jpg': ('Moring

In [81]:
total_plant_class_tally

{'Artocarpus Heterophyllus (Jackfruit)': {'Moringa Oleifera (Malunggay)': 34,
  'Vitex Negundo (Lagundi)': 3,
  'Mangifera Indica (Mango)': 3,
  'Blumea Balsamifera (Sambong)': 2,
  'Artocarpus Heterophyllus (Jackfruit)': 4,
  'Momordica Charantia (Ampalaya)': 4,
  'Psidium Guajava (Guava)': 1},
 'Blumea Balsamifera (Sambong)': {'Blumea Balsamifera (Sambong)': 8,
  'Moringa Oleifera (Malunggay)': 29,
  'Vitex Negundo (Lagundi)': 5,
  'Mangifera Indica (Mango)': 5,
  'Mentha (Mint)': 2,
  'Psidium Guajava (Guava)': 1,
  'Momordica Charantia (Ampalaya)': 2},
 'Citrus Limon (Lemon)': {'Blumea Balsamifera (Sambong)': 1,
  'Moringa Oleifera (Malunggay)': 42,
  'Artocarpus Heterophyllus (Jackfruit)': 1,
  'Mangifera Indica (Mango)': 2,
  'Psidium Guajava (Guava)': 1,
  'Mentha (Mint)': 1,
  'Momordica Charantia (Ampalaya)': 2},
 'Jasminum (Jasmine)': {'Mangifera Indica (Mango)': 9,
  'Jasminum (Jasmine)': 7,
  'Moringa Oleifera (Malunggay)': 3,
  'Momordica Charantia (Ampalaya)': 24,
  'Vite

In [82]:
accuracies = {}

for key, value in total_plant_class_tally.items():
    total = 0 

    for _k, _v in value.items():
        total += _v

    if key in value:
        accuracies[key] = value[key] / total 
    else:
        accuracies[key] = 0

accuracies
        


{'Artocarpus Heterophyllus (Jackfruit)': 0.0784313725490196,
 'Blumea Balsamifera (Sambong)': 0.15384615384615385,
 'Citrus Limon (Lemon)': 0,
 'Jasminum (Jasmine)': 0.1346153846153846,
 'Mangifera Indica (Mango)': 0.09615384615384616,
 'Mentha (Mint)': 0.08163265306122448,
 'Momordica Charantia (Ampalaya)': 0.08163265306122448,
 'Moringa Oleifera (Malunggay)': 0.7323943661971831,
 'Psidium Guajava (Guava)': 0,
 'Vitex Negundo (Lagundi)': 0.2549019607843137}

In [83]:
total_acc = 0
for _k, _v in accuracies.items():
    total_acc += _v 

In [84]:
total_acc

1.61360839026835